In [1]:
# !pip install transformers
# !pip install datasets
import os
import re
import soundfile as sf
import torch
import numpy as np
from datasets import load_metric, Dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import pandas as pd
import subprocess
from pydub import AudioSegment

In [2]:
project_path = os.path.abspath(os.path.join(os.path.abspath("."), ".."))

In [6]:
DATA_PATH = "K:\\AIPI540\\Individual Project\\cv-corpus-8.0-2022-01-19-en\\cv-corpus-8.0-2022-01-19\\en\\clips"
LABEL_PATH = "K:\\AIPI540\\Individual Project\\cv-corpus-8.0-2022-01-19-en\\cv-corpus-8.0-2022-01-19\\en"
AUDIO_BASE = "E:\\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\Data\\Train"
VOCAL_PATH = "E:\\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\Data\\Train\\Vocal"
CLEAN_AUDIO = "E:\\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\Data\\Train\\Clean"
SIZE = 20000
SAMPLE_RATE = 16000

In [6]:
def train_test_split(ds, test_size = 0.3):
    n = len(ds)
    idx = np.random.permutation(n)
    train = ds.select(idx[round(n*test_size):])
    test = ds.select(idx[:round(n*test_size)])
    return train, test

def label_preprocess(line):
    line['path'] = os.path.join(CLEAN_AUDIO, line['path'].split(".")[0]+".flac")
    line['sentence'] = re.sub(r'[^A-Za-z0-9 ]+',"",line['sentence']).upper()
    return line
# labels = pd.read_csv(os.path.join(LABEL_PATH, "train.tsv"), sep = "\t").loc[:SIZE, ['path', 'sentence']]
# processed_labels = labels.copy().apply(label_preprocess, axis = 1)
# processed_labels.head()
    
def normalize(audio_path, audio_file, output_path, format = "wav"):
    try:
        audio_name = audio_file.split(".")[0]
        input_path = os.path.join(audio_path, audio_file)
        output_path = f"{os.path.join(output_path, audio_name)}-normalized.{format}"
        cmd = f'ffmpeg-normalize \"{input_path}\" -o \"{output_path}\"'
        subprocess.call(cmd)
    except Exception as ex:
        print("Error: ", ex)

def vocal_spearation(audio_path, output_path):
    try:
        cmd = f'python inference.py --input \"{audio_path}\" --output \"{output_path}\" --gpu 0 -B 4'
        subprocess.call(cmd)
    except Exception as ex:
        print("Error: ", ex)

def trans2flac(audio_path, audio_name, output_path, format):
    audio_type = audio_name.split(".")[-1][1:]
    exp_file = audio_name.split(".")[0]
    audio = AudioSegment.from_file(os.path.join(audio_path, audio_name), format=audio_type)
    audio.export(f'{os.path.join(output_path, exp_file)}.{format}', format = str(format))
    return f"{exp_file}.{format}"

# for idx, row in data.iterrows():
#     trans2flac(VOCAL_PATH, row['path'], CLEAN_AUDIO, 'flac')

def audio_preprocess(audio_file):
    audio_name = audio_file.split('.')[0]
    audio_format = 'wav'

    nor_sep_folder = os.path.join(AUDIO_BASE, "Normalized")
    if not os.path.exists(nor_sep_folder):
        os.mkdir(nor_sep_folder)

    print("Normalizing...")
    normalize(DATA_PATH, audio_file, nor_sep_folder, audio_format)
    print("Done")

    print("Separating...")
    normalized_name = f"{audio_name}-normalized.{audio_format}"
    audio_path = os.path.join(nor_sep_folder, normalized_name)
    vocal_spearation(audio_path, VOCAL_PATH)
    print("Done")
# labels['path'].apply(audio_preprocess)

def prepare_dataset(batch):
    # audio = batch["audio"]
    audio_input, sample_rate = sf.read(batch["path"])
    # batched output is "un-batched"
    batch["input_values"] = processor(audio_input, sampling_rate=sample_rate).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

def extract_audio(path, export_path = "E:\\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\Data\\Train\\Normalized", format = "wav"):
    try:
        file_name = path.split("\\")[-1]
        audio_name = file_name.split(".")[0]
        audio_path = os.path.join(export_path, audio_name) + f".{format}"
        cmd = f'ffmpeg -i \"{path}\" -f \"{format}\" -vn -ac 1 -ar {SAMPLE_RATE} -y \"{audio_path}\"'
        subprocess.call(cmd)
    except Exception as ex:
        print("Error: ", ex)

In [7]:
labels = pd.read_csv(os.path.join(LABEL_PATH, "train.tsv"), sep = "\t").loc[:SIZE, ['path', 'sentence']]
processed_labels = labels.copy().apply(label_preprocess, axis = 1)
processed_labels.head()

NameError: name 'label_preprocess' is not defined

In [8]:
processed_labels.to_csv("E:\\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\Data\\Train\\labels.csv", index=False)

In [45]:
processed_labels.path = processed_labels['path'].apply(lambda x: os.path.join(DATA_PATH, x))
processed_labels.head()

,path,sentence
0,K:\AIPI540\Individual Project\cv-corpus-8.0-20...,THEREAFTER THE CLASS WAS HIGHLY RESPECTED
1,K:\AIPI540\Individual Project\cv-corpus-8.0-20...,BANARAS HINDU UNIVERSITY IS A CENTRAL UNIVERSI...
2,K:\AIPI540\Individual Project\cv-corpus-8.0-20...,ON DISPLAY ARE HOME FURNISHINGS PIONEER TOOLS ...
3,K:\AIPI540\Individual Project\cv-corpus-8.0-20...,ELEVA AND STRUM EACH HOUSE AN ELEMENTARY SCHOOL
4,K:\AIPI540\Individual Project\cv-corpus-8.0-20...,THE EASTERN PORTION OF THE COUNTY LIES WITHIN ...


In [46]:
processed_labels['path'][0]

'K:\\AIPI540\\Individual Project\\cv-corpus-8.0-2022-01-19-en\\cv-corpus-8.0-2022-01-19\\en\\clips\\common_voice_en_28449980.mp3'

In [1]:
import pandas as pd
labels = pd.read_csv("E:\\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\Data\\Train\\labels.csv")

In [8]:
print(labels.loc[:4, :])

                           path  \
0  common_voice_en_28449980.mp3   
1  common_voice_en_28449981.mp3   
2  common_voice_en_28449984.mp3   
3  common_voice_en_28449986.mp3   
4  common_voice_en_20293200.mp3   

                                            sentence  
0         Thereafter the class was highly respected.  
1  Banaras Hindu University is a Central Universi...  
2  On display are home furnishings, pioneer tools...  
3   Eleva and Strum each house an elementary school.  
4  The eastern portion of the county lies within ...  


In [49]:
processed_labels['path'].apply(extract_audio)

0        None
1        None
2        None
3        None
4        None
         ... 
19996    None
19997    None
19998    None
19999    None
20000    None
Name: path, Length: 20001, dtype: object

In [ ]:
import pandas as pd
from pydub import AudioSegment
def trans2flac(audio_path, audio_name, output_path, format):
    audio_type = 'wav'
    exp_file = audio_name.split(".")[0]
    audio = AudioSegment.from_file(os.path.join(audio_path, exp_file + '.wav'), format=audio_type)
    audio.export(f'{os.path.join(output_path, exp_file)}.{format}', format = str(format))
    return f"{exp_file}.{format}"
LABEL_PATH = "K:\\AIPI540\\Individual Project\\cv-corpus-8.0-2022-01-19-en\\cv-corpus-8.0-2022-01-19\\en"
SIZE = 20000
labels = pd.read_csv(os.path.join(LABEL_PATH, "train.tsv"), sep = "\t").loc[:SIZE, ['path', 'sentence']]
CLEAN_AUDIO = "E:\\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\Data\\Train\\Clean"
normal_path = "E:\\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\Data\\Train\\Normalized"
labels.path.apply(lambda x: trans2flac(normal_path, x, CLEAN_AUDIO, 'flac'))

In [ ]:
processed_labels.path = labels.path.apply(lambda x: os.path.join(CLEAN_AUDIO, x))

In [5]:
label = pd.read_csv(os.path.join(AUDIO_BASE, "label.csv"))

In [5]:
label.path = label['path'].apply(lambda x: os.path.join(CLEAN_AUDIO, x))
label.head()

,path,sentence
0,E:\Graduate\2021-2022 Term 2\AIPI540\Individua...,THEREAFTER THE CLASS WAS HIGHLY RESPECTED
1,E:\Graduate\2021-2022 Term 2\AIPI540\Individua...,BANARAS HINDU UNIVERSITY IS A CENTRAL UNIVERSI...
2,E:\Graduate\2021-2022 Term 2\AIPI540\Individua...,ON DISPLAY ARE HOME FURNISHINGS PIONEER TOOLS ...
3,E:\Graduate\2021-2022 Term 2\AIPI540\Individua...,ELEVA AND STRUM EACH HOUSE AN ELEMENTARY SCHOOL
4,E:\Graduate\2021-2022 Term 2\AIPI540\Individua...,THE EASTERN PORTION OF THE COUNTY LIES WITHIN ...


In [6]:
dataset = Dataset.from_pandas(label)

In [7]:
# load pretrained model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to("cuda")
model.freeze_feature_encoder()

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
train, test = train_test_split(dataset)
train_pro = train.map(prepare_dataset, remove_columns=train.column_names)
test_pro = test.map(prepare_dataset, remove_columns=test.column_names)

100%|██████████| 1845/1845 [00:04<00:00, 410.08ex/s]


In [9]:
repo_name = "E:\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\wav2vec2-base-960h-finetune"

In [10]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=2,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=150,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=5,
  push_to_hub=False,
)

In [12]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [13]:
wer_metric = load_metric("wer")

In [14]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_pro,
    eval_dataset=test_pro,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


In [16]:
trainer.train()

D:\14183\ProgramData\Anaconda3\envs\ind\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4306
  Num Epochs = 150
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 161400
  0%|          | 400/161400 [02:34<11:38:46,  3.84it/s]***** Running Evaluation *****
  Num examples = 1845
  Batch size = 8


{'loss': 115.4869, 'learning_rate': 0.00023459999999999998, 'epoch': 0.37}


                                                       
  0%|          | 400/161400 [04:17<11:38:46,  3.84it/s]Saving model checkpoint to E:\Graduate\2021-2022 Term 2\AIPI540\Individual Project\wav2vec2-base-960h-finetune\checkpoint-400
Configuration saved in E:\Graduate\2021-2022 Term 2\AIPI540\Individual Project\wav2vec2-base-960h-finetune\checkpoint-400\config.json


{'eval_loss': 550.9115600585938, 'eval_wer': 0.5642275551746304, 'eval_runtime': 103.1518, 'eval_samples_per_second': 17.886, 'eval_steps_per_second': 2.239, 'epoch': 0.37}


Model weights saved in E:\Graduate\2021-2022 Term 2\AIPI540\Individual Project\wav2vec2-base-960h-finetune\checkpoint-400\pytorch_model.bin
Feature extractor saved in E:\Graduate\2021-2022 Term 2\AIPI540\Individual Project\wav2vec2-base-960h-finetune\checkpoint-400\preprocessor_config.json
Deleting older checkpoint [E:\Graduate\2021-2022 Term 2\AIPI540\Individual Project\wav2vec2-base-960h-finetune\checkpoint-800] due to args.save_total_limit
  0%|          | 800/161400 [06:48<12:38:29,  3.53it/s]  ***** Running Evaluation *****
  Num examples = 1845
  Batch size = 8


...

                                                          
  5%|▌         | 8800/161400 [1:32:09<12:45:53,  3.32it/s]Saving model checkpoint to E:\Graduate\2021-2022 Term 2\AIPI540\Individual Project\wav2vec2-base-960h-finetune\checkpoint-8800
Configuration saved in E:\Graduate\2021-2022 Term 2\AIPI540\Individual Project\wav2vec2-base-960h-finetune\checkpoint-8800\config.json


{'eval_loss': 547.8412475585938, 'eval_wer': 0.6310263552603386, 'eval_runtime': 95.5429, 'eval_samples_per_second': 19.311, 'eval_steps_per_second': 2.418, 'epoch': 8.18}


Model weights saved in E:\Graduate\2021-2022 Term 2\AIPI540\Individual Project\wav2vec2-base-960h-finetune\checkpoint-8800\pytorch_model.bin
Feature extractor saved in E:\Graduate\2021-2022 Term 2\AIPI540\Individual Project\wav2vec2-base-960h-finetune\checkpoint-8800\preprocessor_config.json
Deleting older checkpoint [E:\Graduate\2021-2022 Term 2\AIPI540\Individual Project\wav2vec2-base-960h-finetune\checkpoint-6800] due to args.save_total_limit
  6%|▌         | 9200/161400 [1:35:04<16:13:46,  2.60it/s]  ***** Running Evaluation *****
  Num examples = 1845
  Batch size = 8


{'loss': 126.5893, 'learning_rate': 0.00028380484773151023, 'epoch': 8.55}


MemoryError: 